In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.max_rows', None)     
pd.set_option('display.float_format', '{:.4f}'.format)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans

In [2]:
df_players = pd.read_csv('../data/player_stats.csv')
df_player_info = pd.read_csv('../data/players.csv')
df_players.head()

,player_match_stat_id,match_id,player_id,team_id,is_substitute,played_position,jersey_number,market_value_eur_at_match,sofascore_rating,minutes_played,touches,goals,assists,own_goals,passes_accurate,passes_total,passes_key,long_balls_accurate,long_balls_total,crosses_accurate,crosses_total,shots_total,shots_on_target,shots_off_target,shots_blocked_by_opponent,dribbles_successful,dribbles_attempts,possession_lost,dispossessed,duels_won,duels_lost,aerials_won,aerials_lost,ground_duels_won,ground_duels_total,tackles,interceptions,clearances,shots_blocked_by_player,dribbled_past,fouls_committed,fouls_suffered,saves,punches_made,high_claims,saves_inside_box,sweeper_keeper_successful,sweeper_keeper_total,goals_prevented,runs_out_successful,penalties_saved,penalty_committed,expected_goals,expected_assists,penalty_won,penalty_miss,big_chances_missed,errors_leading_to_shot,big_chances_created,errors_leading_to_goal
0,1,11352250,84844,60,f,G,1,1900000.0000,6.4000,90,46,0,0,0,27,39,0,6,18,0,0,0,0,0,0,0,0,12,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,-0.2972,0,0,0,0.0000,0.0053,0,0,0,0,0,0
1,2,11352250,943645,60,f,D,37,11600000.0000,6.9000,75,52,0,0,0,32,36,1,1,1,0,1,0,0,0,0,0,0,6,0,6,6,0,3,0,3,2,0,0,0,1,1,2,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0691,0,0,0,0,0,0
2,3,11352250,1023567,60,f,D,27,37000000.0000,6.5000,90,102,0,0,0,77,96,0,4,15,0,0,0,0,0,0,0,0,20,0,5,8,4,5,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0122,0,0,0,0,0,0
3,4,11352250,830659,60,f,D,25,22000000.0000,6.7000,60,62,0,0,0,43,51,0,3,8,0,0,0,0,0,0,0,0,10,1,3,6,1,2,2,0,2,1,3,1,2,1,0,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0172,0,0,0,0,0,0
4,5,11352250,1097425,60,f,D,3,36000000.0000,6.8000,90,75,0,0,0,31,40,1,2,3,1,4,0,0,0,0,0,0,17,1,10,6,4,2,6,3,2,0,3,0,1,0,3,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0493,0,0,0,0,0,0


In [3]:
df_players = df_players.rename(columns = {'ground_duels_total':'dribble_att'})
df_players['played_position'].value_counts()

played_position
M    55393
D    50547
F    31387
G    16950
Name: count, dtype: int64

In [4]:
df_players['ground_duel_won_match'] = df_players['duels_won'] - df_players['aerials_won']
df_players['ground_duel_lost'] = df_players['duels_lost'] - df_players['aerials_lost']


In [5]:
df_player_with_info = pd.merge(df_players, df_player_info[['player_id', 'name', 'country_name']], on=['player_id'], how = 'left')
df_player_with_info.head()

,player_match_stat_id,match_id,player_id,team_id,is_substitute,played_position,jersey_number,market_value_eur_at_match,sofascore_rating,minutes_played,touches,goals,assists,own_goals,passes_accurate,passes_total,passes_key,long_balls_accurate,long_balls_total,crosses_accurate,crosses_total,shots_total,shots_on_target,shots_off_target,shots_blocked_by_opponent,dribbles_successful,dribbles_attempts,possession_lost,dispossessed,duels_won,duels_lost,aerials_won,aerials_lost,ground_duels_won,dribble_att,tackles,interceptions,clearances,shots_blocked_by_player,dribbled_past,fouls_committed,fouls_suffered,saves,punches_made,high_claims,saves_inside_box,sweeper_keeper_successful,sweeper_keeper_total,goals_prevented,runs_out_successful,penalties_saved,penalty_committed,expected_goals,expected_assists,penalty_won,penalty_miss,big_chances_missed,errors_leading_to_shot,big_chances_created,errors_leading_to_goal,ground_duel_won_match,ground_duel_lost,name,country_name
0,1,11352250,84844,60,f,G,1,1900000.0000,6.4000,90,46,0,0,0,27,39,0,6,18,0,0,0,0,0,0,0,0,12,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,-0.2972,0,0,0,0.0000,0.0053,0,0,0,0,0,0,0,0,Neto,Brazil
1,2,11352250,943645,60,f,D,37,11600000.0000,6.9000,75,52,0,0,0,32,36,1,1,1,0,1,0,0,0,0,0,0,6,0,6,6,0,3,0,3,2,0,0,0,1,1,2,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0691,0,0,0,0,0,0,6,3,Max Aarons,England
2,3,11352250,1023567,60,f,D,27,37000000.0000,6.5000,90,102,0,0,0,77,96,0,4,15,0,0,0,0,0,0,0,0,20,0,5,8,4,5,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0122,0,0,0,0,0,0,1,3,Illia Zabarnyi,NaN
3,4,11352250,830659,60,f,D,25,22000000.0000,6.7000,60,62,0,0,0,43,51,0,3,8,0,0,0,0,0,0,0,0,10,1,3,6,1,2,2,0,2,1,3,1,2,1,0,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0172,0,0,0,0,0,0,2,4,Marcos Senesi,NaN
4,5,11352250,1097425,60,f,D,3,36000000.0000,6.8000,90,75,0,0,0,31,40,1,2,3,1,4,0,0,0,0,0,0,17,1,10,6,4,2,6,3,2,0,3,0,1,0,3,0,0,0,0,0,0,NaN,0,0,0,0.0000,0.0493,0,0,0,0,0,0,6,4,Miloš Kerkez,Hungary


In [6]:
df_gk =df_player_with_info.loc[(df_player_with_info['played_position'] == 'G') & (df_player_with_info['minutes_played'] > 15)]
df_def = df_player_with_info.loc[(df_player_with_info['played_position'] == 'D') & (df_player_with_info['minutes_played'] > 15)]
df_mf = df_player_with_info.loc[(df_player_with_info['played_position'] == 'M') & (df_player_with_info['minutes_played'] > 15)]
df_att = df_player_with_info.loc[(df_player_with_info['played_position'] == 'F') & (df_player_with_info['minutes_played'] > 15)]

# Defenders

In [7]:
N_CLUSTERS = 2
stats_to_sum = ['ground_duel_won_match', 'ground_duel_lost', 'dribble_att', 'passes_key', 'long_balls_accurate',
                 'long_balls_total', 'crosses_accurate', 'duels_won', 'duels_lost', 'crosses_total',
            'aerials_won', 'aerials_lost', 'tackles', 'interceptions',
            'clearances', 'shots_blocked_by_player', 'dribbled_past', 'fouls_committed', 'fouls_suffered', 'minutes_played']

df_player_aggregated_def = df_def.groupby(['player_id', 'name'])[stats_to_sum].sum().reset_index()
min_minutes_threshold = 5 * 90
df_player_filtered_def = df_player_aggregated_def[df_player_aggregated_def['minutes_played'] >= min_minutes_threshold].copy()


df_player_filtered_def.loc[:, 'games_played_equivalent'] = df_player_filtered_def['minutes_played'] / 90

for col in stats_to_sum:

    df_player_filtered_def.loc[:, f'{col}_p90'] = (df_player_filtered_def[col] / df_player_filtered_def['games_played_equivalent']).fillna(0)

#===============================================================================================================================================
feature_columns_for_clustering_def = cols_p90 = [
    'ground_duel_won_match_p90', 'ground_duel_lost_p90', 'passes_key_p90', 'long_balls_accurate_p90', 'long_balls_total_p90',
    'crosses_accurate_p90', 'dribble_att_p90', 'duels_won_p90', 'duels_lost_p90', 'crosses_total_p90',
    'aerials_won_p90', 'aerials_lost_p90', 'tackles_p90', 'interceptions_p90',
    'clearances_p90', 'shots_blocked_by_player_p90', 'dribbled_past_p90', 'fouls_committed_p90',
    'fouls_suffered_p90']

features_df = df_player_filtered_def[feature_columns_for_clustering_def].copy()
#===============================================================================================================================================
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_df)
df_scaled_features = pd.DataFrame(scaled_features, columns=feature_columns_for_clustering_def, index=features_df.index)
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=2025, n_init=10) 
df_player_filtered_def['posicion_cluster'] = kmeans.fit_predict(df_scaled_features)

#===============================================================================================================================================
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_) 
df_centroids = pd.DataFrame(cluster_centers, columns=feature_columns_for_clustering_def)
df_centroids.head()



,ground_duel_won_match_p90,ground_duel_lost_p90,passes_key_p90,long_balls_accurate_p90,long_balls_total_p90,crosses_accurate_p90,dribble_att_p90,duels_won_p90,duels_lost_p90,crosses_total_p90,aerials_won_p90,aerials_lost_p90,tackles_p90,interceptions_p90,clearances_p90,shots_blocked_by_player_p90,dribbled_past_p90,fouls_committed_p90,fouls_suffered_p90
0,3.8920,3.2632,0.8320,1.6736,4.1433,0.6463,1.4517,4.9733,4.3335,2.8835,1.0813,1.0703,2.1521,1.1064,2.1035,0.3003,0.9374,1.1182,0.9755
1,2.1061,1.6214,0.2375,2.8318,6.4284,0.0641,0.3131,4.2655,3.0924,0.3140,2.1594,1.4710,1.4057,1.1991,4.0315,0.7188,0.4812,0.9248,0.5172


In [8]:
cluster_map = {0: 'LB', 1: 'CB'} 
df_player_filtered_def['rol_inferido'] = df_player_filtered_def['posicion_cluster'].map(cluster_map)
df_player_filtered_def[['name', 'rol_inferido']].head(25)

,name,rol_inferido
1,Gerard Piqué,CB
2,James Milner,LB
3,Christian Fuchs,CB
4,Raúl Albiol,CB
5,Dani Alves,LB
7,Gary Cahill,CB
8,Antonio Barragán,LB
10,Phil Jagielka,CB
11,Ashley Young,LB
12,Sergio Ramos,CB


# Middlefilders

In [9]:
N_CLUSTERS = 4
stats_to_sum = [
    'touches', 'passes_accurate', 'passes_total', 'passes_key', 'long_balls_accurate', 'long_balls_total',
    'ground_duel_won_match', 'dribble_att', 'tackles', 'interceptions', 'clearances', 'duels_won', 'duels_lost',
    'aerials_won', 'aerials_lost', 'fouls_committed', 'fouls_suffered',
    'shots_total', 'shots_on_target', 'shots_off_target', 'expected_goals', 'expected_assists',
    'big_chances_created', 'big_chances_missed', 'crosses_accurate', 'crosses_total', 'goals', 'assists',
    'possession_lost', 'dispossessed', 'errors_leading_to_goal', 'minutes_played'
]


df_player_aggregated_mf = df_mf.groupby(['player_id', 'name'])[stats_to_sum].sum().reset_index()
min_minutes_threshold = 5 * 90
df_player_filtered_mf = df_player_aggregated_mf[df_player_aggregated_mf['minutes_played'] >= min_minutes_threshold].copy()


df_player_filtered_mf.loc[:, 'games_played_equivalent'] = df_player_filtered_mf['minutes_played'] / 90

for col in stats_to_sum:
    df_player_filtered_mf.loc[:, f'{col}_p90'] = (df_player_filtered_mf[col] / df_player_filtered_mf['games_played_equivalent']).fillna(0)

#===============================================================================================================================================
feature_columns_for_clustering_mf = selected_variables_p90 = [
    'touches_p90', 'passes_accurate_p90', 'passes_total_p90', 'passes_key_p90', 'long_balls_accurate_p90', 'long_balls_total_p90',
    'tackles_p90', 'interceptions_p90', 'clearances_p90', 'duels_won_p90', 'duels_lost_p90',
    'ground_duel_won_match_p90', 'dribble_att_p90', 'aerials_won_p90', 'aerials_lost_p90', 'fouls_committed_p90',
    'fouls_suffered_p90', 'shots_total_p90', 'shots_on_target_p90', 'shots_off_target_p90', 'expected_goals_p90',
    'expected_assists_p90', 'big_chances_created_p90', 'big_chances_missed_p90', 'crosses_accurate_p90', 'crosses_total_p90',
    'possession_lost_p90', 'dispossessed_p90', 'errors_leading_to_goal_p90', 'goals_p90', 'assists_p90'
]

features_df = df_player_filtered_mf[feature_columns_for_clustering_mf].copy()
#===============================================================================================================================================
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_df)
df_scaled_features = pd.DataFrame(scaled_features, columns=feature_columns_for_clustering_mf, index=features_df.index)
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=2025, n_init=10) 
df_player_filtered_mf['posicion_cluster'] = kmeans.fit_predict(df_scaled_features)

#===============================================================================================================================================
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_) 
df_centroids = pd.DataFrame(cluster_centers, columns=feature_columns_for_clustering_mf)
df_centroids.head()



,touches_p90,passes_accurate_p90,passes_total_p90,passes_key_p90,long_balls_accurate_p90,long_balls_total_p90,tackles_p90,interceptions_p90,clearances_p90,duels_won_p90,duels_lost_p90,ground_duel_won_match_p90,dribble_att_p90,aerials_won_p90,aerials_lost_p90,fouls_committed_p90,fouls_suffered_p90,shots_total_p90,shots_on_target_p90,shots_off_target_p90,expected_goals_p90,expected_assists_p90,big_chances_created_p90,big_chances_missed_p90,crosses_accurate_p90,crosses_total_p90,possession_lost_p90,dispossessed_p90,errors_leading_to_goal_p90,goals_p90,assists_p90
0,54.3552,27.1057,33.6503,1.7145,1.1741,2.1630,1.3071,0.5605,0.5820,4.7758,5.6853,4.2068,3.0661,0.5691,0.9749,0.9808,1.3776,0.0000,0.7667,0.7210,0.1599,0.1255,0.3153,0.1954,0.9456,3.9284,14.9997,1.2940,0.0059,0.2468,0.1943
1,74.4457,49.9992,58.1235,1.2750,3.1060,5.2021,1.8072,0.9488,0.9413,4.5668,4.5071,3.8051,1.3540,0.7617,0.8831,1.1582,1.1821,0.0000,0.3197,0.4188,0.0403,0.0665,0.1826,0.0494,0.6341,2.4272,12.9577,0.8420,0.0092,0.0807,0.1139
2,50.5229,23.2551,29.7085,1.0769,0.8778,1.8693,1.8576,0.7054,0.7178,6.2146,7.2426,5.1433,2.9819,1.0712,1.5756,1.4907,1.7914,0.0000,0.5244,0.5860,0.0851,0.0547,0.1661,0.1409,0.4895,2.2768,14.2185,1.6116,0.0069,0.1335,0.0975
3,55.0962,32.2761,39.2439,0.6572,1.6373,3.2827,2.1606,1.1473,1.4746,5.1615,5.0145,3.8714,1.1775,1.2902,1.2748,1.4301,1.0534,0.0000,0.2064,0.3312,0.0357,0.0310,0.0841,0.0454,0.2464,1.1740,10.8152,0.7637,0.0133,0.0537,0.0467


In [10]:
cluster_map = {0: 'MCO', 1: 'MC', 2: 'MW', 3: 'MCD'} 
df_player_filtered_mf['rol_inferido'] = df_player_filtered_mf['posicion_cluster'].map(cluster_map)
df_player_filtered_mf[['name', 'rol_inferido']].head(50)

,name,rol_inferido
0,James Milner,MC
1,Joaquín,MCO
2,Raúl García,MW
3,João Moutinho,MC
4,Ivan Rakitić,MC
5,Robert Snodgrass,MCD
6,Aaron Lennon,MCD
7,Mark Noble,MC
8,Theo Walcott,MW
9,Alex Oxlade-Chamberlain,MC


In [11]:
MCO = ["Martin Ødegaard", "James Maddison", "João Félix", "Dani Olmo", "Kai Havertz", "Isco", "Mason Mount", "Nabil Fekir", "Phil Foden", "Takefusa Kubo"]
MC = ["Luka Modrić", "Ilkay Gündoğan", "Frenkie de Jong", "Kevin De Bruyne", "Pedri", "Pablo Gavi", "Bruno Fernandes", "Youri Tielemans", "Saúl Ñíguez"]
MCD = ["Rodri", "Aurélien Tchouaméni", "Declan Rice", "Casemiro", "Sergio Busquets", "Thomas Partey", "Wilfred Ndidi", "Florentino Luís", "João Palhinha", "Eduardo Camavinga"]

filtro_MC = (df_player_filtered_mf['name'].isin(MC))
filtro_MCD = (df_player_filtered_mf['name'].isin(MCD)) & (df_player_filtered_mf['rol_inferido'] == 'MCD')
filtro_MCO = (df_player_filtered_mf['name'].isin(MCO)) & (df_player_filtered_mf['rol_inferido'] == 'MCO')
filtro_MW = (df_player_filtered_mf['rol_inferido'] == 'MW')

validate_mc = df_player_filtered_mf.loc[filtro_MC]
#validate_mc[['name', 'rol_inferido']].head(10)
validate_mco = df_player_filtered_mf.loc[filtro_MCO]
#validate_mco[['name', 'rol_inferido']].head(10)
validate_mcd = df_player_filtered_mf.loc[filtro_MCD]
#validate_mcd[['name', 'rol_inferido']].head(10)

validate_mw = df_player_filtered_mf.loc[filtro_MW]
validate_mw[['name', 'rol_inferido']].head(10)

,name,rol_inferido
2,Raúl García,MW
8,Theo Walcott,MW
24,Fabián Orellana,MW
31,Óscar Trejo,MW
35,Takashi Inui,MW
64,Matt Phillips,MW
65,Damián Suárez,MW
97,Andros Townsend,MW
99,Juraj Kucka,MW
107,Alberto Perea,MW


# Strikers

In [12]:
N_CLUSTERS = 2
stats_to_sum = [
        'touches', 'goals', 'assists', 'passes_accurate', 'passes_total',
        'passes_key', 'long_balls_accurate', 'long_balls_total', 'crosses_accurate',
        'crosses_total', 'shots_total', 'shots_on_target', 'shots_off_target',
        'shots_blocked_by_opponent', 'possession_lost', 'duels_won', 
        'duels_lost', 'aerials_won', 'aerials_lost', 'dribble_att', 'dribbled_past',
        'fouls_suffered','expected_goals', 'expected_assists',
        'big_chances_missed', 'errors_leading_to_shot', 'big_chances_created', 'errors_leading_to_goal',
        'ground_duel_won_match', 'ground_duel_lost', 'minutes_played'
]


df_player_aggregated_att = df_att.groupby(['player_id', 'name'])[stats_to_sum].sum().reset_index()
min_minutes_threshold = 5 * 90
df_player_filtered_att = df_player_aggregated_att[df_player_aggregated_att['minutes_played'] >= min_minutes_threshold].copy()


df_player_filtered_att.loc[:, 'games_played_equivalent'] = df_player_filtered_att['minutes_played'] / 90

for col in stats_to_sum:
    df_player_filtered_att.loc[:, f'{col}_p90'] = (df_player_filtered_att[col] / df_player_filtered_att['games_played_equivalent']).fillna(0)

#===============================================================================================================================================
feature_columns_for_clustering_att = [
    'touches_p90', 'goals_p90', 'assists_p90', 'passes_accurate_p90', 'passes_total_p90',
    'passes_key_p90', 'long_balls_accurate_p90', 'long_balls_total_p90', 'crosses_accurate_p90',
    'crosses_total_p90', 'shots_total_p90', 'shots_on_target_p90', 'shots_off_target_p90',
    'shots_blocked_by_opponent_p90', 'possession_lost_p90', 'duels_won_p90',
    'duels_lost_p90', 'aerials_won_p90', 'aerials_lost_p90', 'dribble_att_p90', 'dribbled_past_p90',
    'fouls_suffered_p90', 'expected_goals_p90', 'expected_assists_p90',
    'big_chances_missed_p90', 'errors_leading_to_shot_p90', 'big_chances_created_p90', 'errors_leading_to_goal_p90',
    'ground_duel_won_match_p90', 'ground_duel_lost_p90'
]


features_df = df_player_filtered_att[feature_columns_for_clustering_att].copy()
#===============================================================================================================================================
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_df)
df_scaled_features = pd.DataFrame(scaled_features, columns=feature_columns_for_clustering_att, index=features_df.index)
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=2025, n_init=10) 
df_player_filtered_att['posicion_cluster'] = kmeans.fit_predict(df_scaled_features)

#===============================================================================================================================================
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_) 
df_centroids = pd.DataFrame(cluster_centers, columns=feature_columns_for_clustering_att)
df_centroids.head()



,touches_p90,goals_p90,assists_p90,passes_accurate_p90,passes_total_p90,passes_key_p90,long_balls_accurate_p90,long_balls_total_p90,crosses_accurate_p90,crosses_total_p90,shots_total_p90,shots_on_target_p90,shots_off_target_p90,shots_blocked_by_opponent_p90,possession_lost_p90,duels_won_p90,duels_lost_p90,aerials_won_p90,aerials_lost_p90,dribble_att_p90,dribbled_past_p90,fouls_suffered_p90,expected_goals_p90,expected_assists_p90,big_chances_missed_p90,errors_leading_to_shot_p90,big_chances_created_p90,errors_leading_to_goal_p90,ground_duel_won_match_p90,ground_duel_lost_p90
0,54.2563,0.2465,0.1710,26.7700,33.1379,1.5708,1.0915,1.9014,0.7481,3.1854,0.0000,0.8501,0.7711,0.6539,15.1552,5.4887,6.5479,0.6642,1.2462,3.5676,0.9190,1.6793,0.1482,0.0961,0.1994,0.0110,0.2915,0.0024,4.8245,5.3017
1,35.3441,0.3062,0.0904,14.2987,19.9602,0.8270,0.4160,0.7850,0.1149,0.6728,0.0000,0.8927,0.8664,0.4719,11.6505,5.3105,7.5730,2.3538,3.2461,1.8000,0.5915,1.3680,0.1938,0.0355,0.3567,0.0044,0.1489,0.0010,2.9568,4.3268


In [13]:
cluster_map = {0: 'EX', 1: 'DC'}
df_player_filtered_att['rol_inferido'] = df_player_filtered_att['posicion_cluster'].map(cluster_map)
df_player_filtered_att[['name', 'rol_inferido']].head(50)

,name,rol_inferido
0,Cristiano Ronaldo,DC
2,Karim Benzema,EX
3,Kévin Gameiro,DC
4,Raúl García,DC
9,Theo Walcott,DC
10,Alex Oxlade-Chamberlain,DC
14,Lionel Messi,EX
15,Roberto Soldado,DC
16,David McGoldrick,DC
17,Teemu Pukki,DC


In [14]:
print(df_player_filtered_mf.head())

   player_id           name  touches  passes_accurate  passes_total  \
0        791   James Milner     2580             1684          1977   
1       1749        Joaquín     1507              899          1054   
2       4581    Raúl García      907              351           568   
3       6562  João Moutinho     5723             3964          4569   
4       7635   Ivan Rakitić     6605             4413          5191   

   passes_key  long_balls_accurate  long_balls_total  ground_duel_won_match  \
0          50                   78               129                    111   
1          57                   18                37                     86   
2           9                   21                46                     81   
3         121                  256               400                    286   
4         142                  386               616                    264   

   dribble_att  tackles  interceptions  clearances  duels_won  duels_lost  \
0           30       

In [15]:
player_data = pd.read_csv('../data/players.csv')
player_data.head()

,player_id,name,height_cm,primary_position,country_name
0,78152,Matz Sels,188.0000,G,Belgium
1,1101362,Jesurun Rak-Sakyi,178.0000,M,England
2,1464649,Rachad Fettal,184.0000,F,Morocco
3,362032,Cristian Herrera,189.0000,M,Spain
4,758168,Moussa Niakhaté,190.0000,D,Senegal


In [17]:
df_gk['rol_inferido'] = 'G'
column_taken = ['player_id', 'name', 'rol_inferido'] 

df_all_inferred_roles = pd.concat([
    df_gk[column_taken],
    df_player_filtered_def[column_taken],
    df_player_filtered_mf[column_taken],
    df_player_filtered_att[column_taken]
], axis=0, ignore_index=True)

df_all_inferred_roles.drop_duplicates(subset=['player_id', 'rol_inferido'], inplace=True)
df_aggregated_roles = df_all_inferred_roles.groupby('player_id')['rol_inferido'].apply(lambda x: list(set(x))).reset_index()
df_aggregated_roles.rename(columns={'rol_inferido': 'PosicionesNaturalesCluster'}, inplace=True)

df_full = pd.merge(player_data, df_aggregated_roles, on='player_id', how='left')
df_full.to_csv('../data/player_cluster.csv')
df_full_not_nan = df_full['PosicionesNaturalesCluster'].isna()


df_full.loc[~df_full_not_nan].head()

C:\Users\javie\AppData\Local\Temp\ipykernel_38608\2063471289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gk['rol_inferido'] = 'G'


,player_id,name,height_cm,primary_position,country_name,PosicionesNaturalesCluster
0,78152,Matz Sels,188.0000,G,Belgium,[G]
4,758168,Moussa Niakhaté,190.0000,D,Senegal,[CB]
6,826139,Reiss Nelson,NaN,F,NaN,[EX]
8,560128,Joelinton,NaN,M,NaN,"[MW, DC]"
9,1085180,Brennan Johnson,186.0000,F,Wales,"[DC, MCO]"
